In [5]:
%pip install -q -U trl transformers accelerate peft
%pip install -q datasets bitsandbytes

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

In [2]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [3]:
# Configure Huggingface token if download model from Huggingface
# import getpass
# import os
# # Get the Huggingface Token
# os.environ["HF_TOKEN"] = getpass.getpass()

In [4]:
base_model = "meta-llama/Meta-Llama-3.1-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(base_model, token=os.environ['HF_TOKEN'])

In [5]:
#tokenizer.save_pretrained("llama3.1-base-tokenizer")

In [6]:
def print_tokens_with_ids(txt):
    tokens = tokenizer.tokenize(txt, add_special_tokens=False)
    token_ids = tokenizer.encode(txt, add_special_tokens=False)
    print(list(zip(tokens, token_ids)))
prompt = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Based on the information provided, rewrite the sentence by changing its tense from past to future.<|eot_id|><|start_header_id|>user<|end_header_id|>

She played the piano beautifully for hours and then stopped as it was midnight.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""
print_tokens_with_ids(prompt)

[('<|begin_of_text|>', 128000), ('<|start_header_id|>', 128006), ('system', 9125), ('<|end_header_id|>', 128007), ('ĊĊ', 271), ('Based', 29815), ('Ġon', 389), ('Ġthe', 279), ('Ġinformation', 2038), ('Ġprovided', 3984), (',', 11), ('Ġrewrite', 18622), ('Ġthe', 279), ('Ġsentence', 11914), ('Ġby', 555), ('Ġchanging', 10223), ('Ġits', 1202), ('Ġtense', 43787), ('Ġfrom', 505), ('Ġpast', 3347), ('Ġto', 311), ('Ġfuture', 3938), ('.', 13), ('<|eot_id|>', 128009), ('<|start_header_id|>', 128006), ('user', 882), ('<|end_header_id|>', 128007), ('ĊĊ', 271), ('She', 8100), ('Ġplayed', 6476), ('Ġthe', 279), ('Ġpiano', 27374), ('Ġbeautifully', 32719), ('Ġfor', 369), ('Ġhours', 4207), ('Ġand', 323), ('Ġthen', 1243), ('Ġstopped', 10717), ('Ġas', 439), ('Ġit', 433), ('Ġwas', 574), ('Ġmidnight', 33433), ('.', 13), ('<|eot_id|>', 128009), ('<|start_header_id|>', 128006), ('assistant', 78191), ('<|end_header_id|>', 128007), ('ĊĊ', 271)]


In [9]:

# device_map={"":0} simply means "try to fit the entire model on the device 0" - device 0 in this case would be the GPU-0
base_model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"
#base_model_bnb_4b = AutoModelForCausalLM.from_pretrained(base_model_name, quantization_config=bnb_config, device_map = 'auto', token=os.environ['HF_TOKEN'])
base_model_bnb_4b = AutoModelForCausalLM.from_pretrained("base_model_bnb_4b", device_map="cuda:0")

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [27]:
tokenizer = AutoTokenizer.from_pretrained("base_model_bnb_4b")

In [10]:
base_model_bnb_4b

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps

In [ ]:
#base_model_bnb_4b.save_pretrained("base_model_bnb_4b")

In [11]:
prompt="""<|start_header_id|>system<|end_header_id|>

You are a helpful assistant with access to the following functions.
You are provided with function signatures within <tools></tools> XML tags. You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions. For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:
<tool_call>{"name": <function-name>,"arguments": <args-dict>}</tool_call>
Here are the available tools:
<tools> {
    "name": "calculate_age",
    "description": "Calculate the age based on the date of birth",
    "parameters": {
        "type": "object",
        "properties": {
            "date_of_birth": {
                "type": "string",
                "format": "date",
                "description": "The date of birth"
            }
        },
        "required": [
            "date_of_birth"
        ]
    }
} </tools><|eot_id|><|start_header_id|>user<|end_header_id|>

Hi, can you tell me how old I am if I was born on 1990-05-15?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""
input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
outputs = base_model_bnb_4b.generate(input_ids=input_ids,
                          pad_token_id=tokenizer.eos_token_id,
                          max_new_tokens=200,
                          do_sample=True,
                          top_p=0.9,
                          temperature=0.1)
result = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=False)[0]
print(result)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful assistant with access to the following functions.
You are provided with function signatures within <tools></tools> XML tags. You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions. For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:
<tool_call>{"name": <function-name>,"arguments": <args-dict>}</tool_call>
Here are the available tools:
<tools> {
    "name": "calculate_age",
    "description": "Calculate the age based on the date of birth",
    "parameters": {
        "type": "object",
        "properties": {
            "date_of_birth": {
                "type": "string",
                "format": "date",
                "description": "The date of birth"
            }
        },
        "required": [
            "date_of_birth"
        ]
    }
} </tools><|eot

In [ ]:
prompt="""<|start_header_id|>system<|end_header_id|>

You are a helpful assistant with access to the following functions.
You are provided with function signatures within <tools></tools> XML tags. You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions. For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:
<tool_call>{"name": <function-name>,"arguments": <args-dict>}</tool_call>
Here are the available tools:
<tools> {
    "name": "calculate_age",
    "description": "Calculate the age based on the date of birth",
    "parameters": {
        "type": "object",
        "properties": {
            "date_of_birth": {
                "type": "string",
                "format": "date",
                "description": "The date of birth"
            }
        },
        "required": [
            "date_of_birth"
        ]
    }
} </tools><|eot_id|><|start_header_id|>user<|end_header_id|>

Hi, can you tell me how old I am if I was born on 1990-05-15?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

<tool_call>{"name": "calculate_age","arguments": {"date_of_birth": "1990-05-15"}}</tool_call><|eot_id|><|start_header_id|>ipython<|end_header_id|>

{"result": "{\"age\": 34}"}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""
input_ids = tokenizer(prompt, return_tensors="pt", truncation=False).input_ids.cuda()
outputs = base_model_bnb_4b.generate(input_ids=input_ids,
                          pad_token_id=tokenizer.eos_token_id,
                          max_new_tokens=256,
                          do_sample=True,
                          top_p=0.9,
                          temperature=0.1)
result = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=False)[0]
print(result)

In [ ]:
prompt="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful assistant with access to the following functions.
You are provided with function signatures within <tools></tools> XML tags. You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions. For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:
<tool_call>{"name": <function-name>,"arguments": <args-dict>}</tool_call>
Here are the available tools:
<tools> {
    "name": "calculate_age",
    "description": "Calculate the age based on the date of birth",
    "parameters": {
        "type": "object",
        "properties": {
            "date_of_birth": {
                "type": "string",
                "format": "date",
                "description": "The date of birth"
            }
        },
        "required": [
            "date_of_birth"
        ]
    }
} </tools><|eot_id|><|start_header_id|>user<|end_header_id|>

Hi, can you tell me how old I am if I was born on 1990-05-15?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

<tool_call>{"name": "calculate_age","arguments": {"date_of_birth": "1990-05-15"}}</tool_call><|eot_id|><|start_header_id|>ipython<|end_header_id|>

{"result": "{"age": 34}"}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

The result is that you are 34 years old.<|eot_id|><|start_header_id|>user<|end_header_id|>

Great, thanks! Can you also book a flight for me to New York next week?<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>

"""
input_ids = tokenizer(prompt, return_tensors="pt", truncation=False).input_ids.cuda()
outputs = base_model_bnb_4b.generate(input_ids=input_ids,
                          pad_token_id=tokenizer.eos_token_id,
                          max_new_tokens=256,
                          do_sample=True,
                          top_p=0.9,
                          temperature=0.1)
result = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=False)[0]
print(result)

In [12]:
base_model_bnb_4b.config

LlamaConfig {
  "_name_or_path": "base_model_bnb_4b",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_storage": "uint8",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": false,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": true,
    "load_in_8bit": false,
    "quant_method"

In [13]:
base_model_bnb_4b.config.use_cache = False

In [ ]:
base_model_bnb_4b

In [14]:
import json

def format_sharegpt_conversations(examples):
    conversations = examples["conversations"]
    formatted_texts = []

    for conversation in conversations:
        formatted_text = ""
        system_message = ""
        tools = ""

        # Extract system message and tools if present
        if conversation[0]["from"] == "system":
            system_content = conversation[0]["value"]
            system_parts = system_content.split("Use them if required -\n")
            system_message = system_parts[0].strip()
            if len(system_parts) > 1:
                tools =  system_parts[1].strip()
            conversation = conversation[1:]  # Remove system message from conversation

        # Add system message and tools if present
        if system_message or tools:
            formatted_text += "<|start_header_id|>system<|end_header_id|>\n\n"
            formatted_text += f"{system_message}\n"
            if tools:
                formatted_text += "You are provided with function signatures within <tools></tools> XML tags. "
                formatted_text += "You may call one or more functions to assist with the user query. "
                formatted_text += "Don't make assumptions about what values to plug into functions. "
                formatted_text += "For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:\n"
                formatted_text += "<tool_call>"
                formatted_text += '{"name": <function-name>,"arguments": <args-dict>}'
                formatted_text += "</tool_call>"
                formatted_text += f"Here are the available tools:\n<tools> {tools} </tools>"
            formatted_text += "<|eot_id|>"

        for i, turn in enumerate(conversation):
            role = turn["from"]
            content = turn["value"]
            last = i == len(conversation) - 1

            if role == "human":
                formatted_text += f"<|start_header_id|>user<|end_header_id|>\n\n{content}<|eot_id|>"
                if last:
                    formatted_text += "<|start_header_id|>assistant<|end_header_id|>\n\n"

            elif role == "gpt":
                formatted_text += f"<|start_header_id|>assistant<|end_header_id|>\n\n"
                if '<functioncall>' in content:
                    function_call = content.split('<functioncall>')[1].split('<|endoftext|>')[0].strip()
                    formatted_text += f"<tool_call>{function_call}</tool_call><|eot_id|>"

                else:
                    formatted_text += f"{content.split('<|endoftext|>')[0].strip()}<|eot_id|>"

            elif role == "tool":
                formatted_text += "<|start_header_id|>ipython<|end_header_id|>\n\n"
                formatted_text += f"{json.dumps({'result': content})}<|eot_id|>"
                if last:
                    formatted_text += "<|start_header_id|>assistant<|end_header_id|>\n\n"

        formatted_texts.append(formatted_text.strip())

    return {"formatted_text": formatted_texts}

In [15]:
# Load the dataset
from datasets import load_dataset
dataset = load_dataset("lilacai/glaive-function-calling-v2-sharegpt", split="train")
dataset = dataset.map(format_sharegpt_conversations, batched=True)

In [16]:
print(dataset["formatted_text"][1])

<|start_header_id|>system<|end_header_id|>

You are a helpful assistant with access to the following functions.
You are provided with function signatures within <tools></tools> XML tags. You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions. For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:
<tool_call>{"name": <function-name>,"arguments": <args-dict>}</tool_call>Here are the available tools:
<tools> {
    "name": "calculate_age",
    "description": "Calculate the age based on the date of birth",
    "parameters": {
        "type": "object",
        "properties": {
            "date_of_birth": {
                "type": "string",
                "format": "date",
                "description": "The date of birth"
            }
        },
        "required": [
            "date_of_birth"
        ]
    }
} </tools><|eot_id|><|start_heade

In [17]:
from peft import LoraConfig
# https://huggingface.co/docs/peft/main/en/conceptual_guides/lora
lora_alpha = 16
lora_dropout = 0.1
lora_r = 32

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    # target_modules=[
    #     "query_key_value",
    #     "dense",
    #     "dense_h_to_4h",
    #     "dense_4h_to_h",
    # ]
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
)

In [18]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

def count_trainable_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [19]:
print_trainable_parameters(base_model_bnb_4b)
count_trainable_params(base_model_bnb_4b)

trainable params: 1050939392 || all params: 4540600320 || trainable%: 23.145384264959926


1050939392

In [20]:
from peft import get_peft_config, get_peft_model, TaskType

peft_model = get_peft_model(base_model_bnb_4b, peft_config)
peft_model.print_trainable_parameters()

trainable params: 83,886,080 || all params: 8,114,147,328 || trainable%: 1.0338


In [21]:
lora_alpha = 16
lora_dropout = 0.1
lora_r = 16

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
)

In [22]:
# 3. Wrap the model with LoRA
peft_model = get_peft_model(base_model_bnb_4b, peft_config)
peft_model.print_trainable_parameters()

trainable params: 41,943,040 || all params: 8,072,204,288 || trainable%: 0.5196


In [23]:
peft_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): 

In [24]:
from trl import SFTConfig
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 2
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 50
logging_steps = 5
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 100
warmup_ratio = 0.03
lr_scheduler_type = "linear"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    gradient_checkpointing=True,
)

sft_config = SFTConfig(
    dataset_text_field="formatted_text",
    max_seq_length=512,
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    gradient_checkpointing=True,
)

In [28]:
from trl import SFTTrainer
trainer = SFTTrainer(
    model=base_model_bnb_4b,
    train_dataset=dataset,
    peft_config=peft_config,
    args=sft_config,
)

Map:   0%|          | 0/112960 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs


In [ ]:
# from trl import SFTTrainer

# trainer = SFTTrainer(
#     model=peft_model,
#     train_dataset=dataset,
#     dataset_text_field="formatted_text",
#     #peft_config=peft_config,
#     args=training_arguments,
# )

In [ ]:
# for name, param in peft_model.named_parameters():
#     print(f"name:{name}\nparam:{param}")

In [ ]:
import torch
device_name = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device = torch.device(device_name)

In [ ]:
import gc
gc.collect()

In [29]:
trainer_stats = trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss
5,1.377600
10,0.931500
15,0.641300
20,0.654300
25,0.420600
30,0.242600
35,0.214400
40,0.159400
45,0.282200
50,0.195100


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


In [30]:
# Get the PEFT model
lora_model = trainer.model

# Merge the LoRA weights with the base model
# merged_model = peft_model.merge_and_unload()

# Save the merged model
lora_model.save_pretrained("lora_model")

# Don't forget to save the tokenizer if you need it
tokenizer = trainer.tokenizer
tokenizer.save_pretrained("lora_model")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

In [35]:
lora_model.config

LlamaConfig {
  "_name_or_path": "base_model_bnb_4b",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_storage": "uint8",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": false,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": true,
    "load_in_8bit": false,
    "quant_method"

In [31]:
lora_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): 

In [37]:
import gc
import torch

# Delete specific large objects
# del some_large_object

# Clear all variables (uncomment if needed)
# %reset

# Garbage collect
gc.collect()

# Empty PyTorch CUDA cache
torch.cuda.empty_cache()

# Reset PyTorch CUDA device
torch.cuda.reset_peak_memory_stats()
torch.cuda.reset_accumulated_memory_stats()

# Reinitialize CUDA device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()
torch.cuda.ipc_collect()
torch.cuda.init()

# Print memory summary
print(torch.cuda.memory_summary(device=device))

print("GPU memory cleanup completed.")

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   8505 MiB |   8505 MiB |      0 B   |      0 B   |
|       from large pool |   8263 MiB |   8263 MiB |      0 B   |      0 B   |
|       from small pool |    241 MiB |    241 MiB |      0 B   |      0 B   |
|---------------------------------------------------------------------------|
| Active memory         |   8505 MiB |   8505 MiB |      0 B   |      0 B   |
|       from large pool |   8263 MiB |   8263 MiB |      0 B   |

In [60]:
base_model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    token=os.environ['HF_TOKEN']
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device device because they were offloaded to the cpu and disk.


In [61]:
base_model.config

LlamaConfig {
  "_name_or_path": "meta-llama/Meta-Llama-3.1-8B-Instruct",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": {
    "factor": 8.0,
    "high_freq_factor": 4.0,
    "low_freq_factor": 1.0,
    "original_max_position_embeddings": 8192,
    "rope_type": "llama3"
  },
  "rope_theta": 500000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.44.2",
  "use_cache": true,
  "vocab_size": 128256
}

In [62]:
base_model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (n

In [63]:
trainer.model.config

LlamaConfig {
  "_name_or_path": "meta-llama/Meta-Llama-3.1-8B-Instruct",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_storage": "uint8",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": false,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": true,
    "load_in_8bit": false

In [64]:
trainer.model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
              (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
              (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
              (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
              (rotary_emb): LlamaRotaryEmbedding()
            )
            (mlp): LlamaMLP(
              (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
              (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
              (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
              (act_fn): SiLU()
     

In [65]:
lora_adapters_path = "lora-adaptor"
trainer.save_model(adapters_path)

In [67]:
for name, _ in base_model.named_modules():
    print(name)


model
model.embed_tokens
model.layers
model.layers.0
model.layers.0.self_attn
model.layers.0.self_attn.q_proj
model.layers.0.self_attn.q_proj.base_layer
model.layers.0.self_attn.q_proj.lora_dropout
model.layers.0.self_attn.q_proj.lora_dropout.default
model.layers.0.self_attn.q_proj.lora_A
model.layers.0.self_attn.q_proj.lora_A.default
model.layers.0.self_attn.q_proj.lora_B
model.layers.0.self_attn.q_proj.lora_B.default
model.layers.0.self_attn.q_proj.lora_embedding_A
model.layers.0.self_attn.q_proj.lora_embedding_B
model.layers.0.self_attn.q_proj.lora_magnitude_vector
model.layers.0.self_attn.k_proj
model.layers.0.self_attn.k_proj.base_layer
model.layers.0.self_attn.k_proj.lora_dropout
model.layers.0.self_attn.k_proj.lora_dropout.default
model.layers.0.self_attn.k_proj.lora_A
model.layers.0.self_attn.k_proj.lora_A.default
model.layers.0.self_attn.k_proj.lora_B
model.layers.0.self_attn.k_proj.lora_B.default
model.layers.0.self_attn.k_proj.lora_embedding_A
model.layers.0.self_attn.k_pro

In [71]:
for name, _ in pretrain_model.named_modules():
    print(name)


model
model.embed_tokens
model.layers
model.layers.0
model.layers.0.self_attn
model.layers.0.self_attn.q_proj
model.layers.0.self_attn.k_proj
model.layers.0.self_attn.v_proj
model.layers.0.self_attn.o_proj
model.layers.0.self_attn.rotary_emb
model.layers.0.mlp
model.layers.0.mlp.gate_proj
model.layers.0.mlp.up_proj
model.layers.0.mlp.down_proj
model.layers.0.mlp.act_fn
model.layers.0.input_layernorm
model.layers.0.post_attention_layernorm
model.layers.1
model.layers.1.self_attn
model.layers.1.self_attn.q_proj
model.layers.1.self_attn.k_proj
model.layers.1.self_attn.v_proj
model.layers.1.self_attn.o_proj
model.layers.1.self_attn.rotary_emb
model.layers.1.mlp
model.layers.1.mlp.gate_proj
model.layers.1.mlp.up_proj
model.layers.1.mlp.down_proj
model.layers.1.mlp.act_fn
model.layers.1.input_layernorm
model.layers.1.post_attention_layernorm
model.layers.2
model.layers.2.self_attn
model.layers.2.self_attn.q_proj
model.layers.2.self_attn.k_proj
model.layers.2.self_attn.v_proj
model.layers.2.

In [70]:
from peft import PeftModel

# Initialize PeftModel with base model and adapters path
new_model = PeftModel.from_pretrained(pretrain_model, lora_adapters_path)

# Merge the trainer adapter with the base model and unload the adapter
new_model = new_model.merge_and_unload()

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:336: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [32]:
prompt="""<|start_header_id|>system<|end_header_id|>

You are a helpful assistant with access to the following functions.
You are provided with function signatures within <tools></tools> XML tags. You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions. For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:
<tool_call>{"name": <function-name>,"arguments": <args-dict>}</tool_call>
Here are the available tools:
<tools> {
    "name": "calculate_age",
    "description": "Calculate the age based on the date of birth",
    "parameters": {
        "type": "object",
        "properties": {
            "date_of_birth": {
                "type": "string",
                "format": "date",
                "description": "The date of birth"
            }
        },
        "required": [
            "date_of_birth"
        ]
    }
} </tools><|eot_id|><|start_header_id|>user<|end_header_id|>

Hi, can you tell me how old I am if I was born on 1990-05-15?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""
input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
outputs = lora_model.generate(input_ids=input_ids,
                          pad_token_id=tokenizer.eos_token_id,
                          max_new_tokens=200,
                          do_sample=True,
                          top_p=0.9,
                          temperature=0.1)
result = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=False)[0]
print(result)

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful assistant with access to the following functions.
You are provided with function signatures within <tools></tools> XML tags. You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions. For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:
<tool_call>{"name": <function-name>,"arguments": <args-dict>}</tool_call>
Here are the available tools:
<tools> {
    "name": "calculate_age",
    "description": "Calculate the age based on the date of birth",
    "parameters": {
        "type": "object",
        "properties": {
            "date_of_birth": {
                "type": "string",
                "format": "date",
                "description": "The date of birth"
            }
        },
        "required": [
            "date_of_birth"
        ]
    }
} </tools><|eot

In [34]:
prompt="""<|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 25 Aug 2024

You are a helpful assistant with access to the following functions.
You are provided with function signatures within <tools></tools> XML tags. You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions. For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:
<tool_call>{"name": <function-name>,"arguments": <args-dict>}</tool_call>
Here are the available tools:
<tools> {
    "name": "calculate_age",
    "description": "Calculate the age based on the date of birth",
    "parameters": {
        "type": "object",
        "properties": {
            "date_of_birth": {
                "type": "string",
                "format": "date",
                "description": "The date of birth"
            }
        },
        "required": [
            "date_of_birth"
        ]
    }
} </tools><|eot_id|><|start_header_id|>user<|end_header_id|>

Hi, can you tell me how old I am if I was born on 1990-05-15?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

<tool_call>{"name": "calculate_age","arguments": {"date_of_birth": "1990-05-15"}}</tool_call><|eot_id|><|start_header_id|>ipython<|end_header_id|>

{"result": "{\"age\": 34}"}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""
input_ids = tokenizer(prompt, return_tensors="pt", truncation=False).input_ids.cuda()
outputs = lora_model.generate(input_ids=input_ids,
                          pad_token_id=tokenizer.eos_token_id,
                          max_new_tokens=256,
                          do_sample=True,
                          top_p=0.9,
                          temperature=0.1)
result = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=False)[0]
print(result)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 25 Aug 2024

You are a helpful assistant with access to the following functions.
You are provided with function signatures within <tools></tools> XML tags. You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions. For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:
<tool_call>{"name": <function-name>,"arguments": <args-dict>}</tool_call>
Here are the available tools:
<tools> {
    "name": "calculate_age",
    "description": "Calculate the age based on the date of birth",
    "parameters": {
        "type": "object",
        "properties": {
            "date_of_birth": {
                "type": "string",
                "format": "date",
                "description": "The date of birth"
            }
        },
        "required"

In [40]:
prompt="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Aug 2024

You are a helpful assistant with access to the following functions.
You are provided with function signatures within <tools></tools> XML tags. You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions. For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:
<tool_call>{"name": <function-name>,"arguments": <args-dict>}</tool_call>
Here are the available tools:
<tools> {
    "name": "calculate_age",
    "description": "Calculate the age based on the date of birth",
    "parameters": {
        "type": "object",
        "properties": {
            "date_of_birth": {
                "type": "string",
                "format": "date",
                "description": "The date of birth"
            }
        },
        "required": [
            "date_of_birth"
        ]
    }
} </tools><|eot_id|><|start_header_id|>user<|end_header_id|>

Hi, can you tell me how old I am if I was born on 1990-05-15?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

<tool_call>{"name": "calculate_age","arguments": {"date_of_birth": "1990-05-15"}}</tool_call><|eot_id|><|start_header_id|>ipython<|end_header_id|>

{"result": "{"age": 34}"}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Please note that the result is an approximation and the actual age may vary depending on the time of birth and the current date.<|eot_id|><|start_header_id|>user<|end_header_id|>

Great, thanks! Can you also book a flight for me to New York next week?<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>

"""
input_ids = tokenizer(prompt, return_tensors="pt", truncation=False).input_ids.cuda()
outputs = new_model.generate(input_ids=input_ids,
                          pad_token_id=tokenizer.eos_token_id,
                          max_new_tokens=256,
                          do_sample=True,
                          top_p=0.9,
                          temperature=0.1)
result = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=False)[0]
print(result)

<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Aug 2024

You are a helpful assistant with access to the following functions.
You are provided with function signatures within <tools></tools> XML tags. You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions. For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:
<tool_call>{"name": <function-name>,"arguments": <args-dict>}</tool_call>
Here are the available tools:
<tools> {
    "name": "calculate_age",
    "description": "Calculate the age based on the date of birth",
    "parameters": {
        "type": "object",
        "properties": {
            "date_of_birth": {
                "type": "string",
                "format": "date",
                "description": "The date of birth"
            }
        },
 

In [45]:

new_model.push_to_hub("syu-hf/llama3.1-instruct-bnb-4bit-tools-ft", token=os.environ['HF_TOKEN'])
tokenizer.push_to_hub("syu-hf/llama3.1-instruct-bnb-4bit-tools-ft", token=os.environ['HF_TOKEN'])


model-00002-of-00002.safetensors:   0%|          | 0.00/3.15G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/syu-hf/llama3.1-instruct-bnb-4bit-tools-ft/commit/3bdf4069a53393649b0a6aafd23d2f606aaaef48', commit_message='Upload LlamaForCausalLM', commit_description='', oid='3bdf4069a53393649b0a6aafd23d2f606aaaef48', pr_url=None, pr_revision=None, pr_num=None)

In [46]:
new_model.config

LlamaConfig {
  "_name_or_path": "meta-llama/Meta-Llama-3.1-8B-Instruct",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_storage": "uint8",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": false,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": true,
    "load_in_8bit": false

In [47]:
new_model.save_pretrained("./llama3.1-instruct-bnb-4bit-tools-ft")
tokenizer.save_pretrained("./llama3.1-instruct-bnb-4bit-tools-ft")